<a href="https://colab.research.google.com/github/Mayugumiii/UIT_DS_Challenge_Text_Reranking/blob/main/UIT_DS_Challenge_Text_Reranking_T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data = pd.read_json('/content/drive/MyDrive/data.json')

data.head()

,context,claim,verdict,evidence
0,Từ cơn sốt rét nam bệnh nhân suýt chết vì vi k...,"Ngày 18/9, theo thông tin từ Bệnh viện Nội tiế...",Refuted,Nhày 18/8 theo thông tin từ Bệnh viện Nội tiết...
1,Chó cắn vào má bé trai 6 tuổi tử vong. Bị chó ...,Cháu H.P.Đ tử vong vì bị chó cắn vào má,Supported,Chó cắn vào má bé trai 6 tuổi tử vong.
2,Thận lợn sống khỏe trong cơ thể người. MỸ - Th...,Thận bò hoạt động trong cơ thể ông Maurice suố...,Refuted,MỸ - Thận lợn hoạt động trong cơ thể ông Mauri...
3,Nha sĩ cảnh báo một điều không nên làm khi đán...,Bạn không nên cho kem đánh răng lên bàn chải ...,Neutral,Bạn không nên cho kem đánh răng lên bàn chải c...
4,Các địa điểm cấm hút thuốc lá: Cần biết để khô...,"Không được hút thuốc tại nhà khách, nhà nghỉ.",Supported,Các địa điểm cấm hút thuốc lá: Cần biết để khô...


In [ ]:
!pip install transformers
!pip install -U sentence-transformers
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1a1e12a485a499c6856b1ebb1b6d5a97f9cc07f0dee482d60d376e0afcefc808
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 27.0 MB/s eta 0

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Load data
df = pd.read_excel('/content/drive/MyDrive/Dataset/output.xlsx')
passages = df.iloc[:500, 0]
labels = df.iloc[:500,1]
claims = df.iloc[:500, 2]

In [ ]:
# Load the PhoBERT model and tokenizer
model = AutoModel.from_pretrained("bert-base-multilingual-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [ ]:
import json

# Create a list of dictionaries to store the data for each claim
output_data = []
i = 0
for passage, label, claim in zip(passages, labels, claims):
    premise_sentences = sent_tokenize(passage)

    # Tokenize the premise sentences and claim
    encoded_premise = [tokenizer(sentence, return_tensors='pt') for sentence in premise_sentences]
    claim = str(claim)  # Convert claim to a string if it's not already
    encoded_claim = tokenizer(claim, return_tensors='pt')


    # Embed the premise sentences and claim
    embedded_premise = [model(**sentence) for sentence in encoded_premise]
    embedded_claim = model(**encoded_claim)

    # Initialize variables for each iteration
    highest_similarity = -1.1
    best_premise_sentence = None

    # Calculate cosine similarity and find the best match
    for sentence, premise_embedding in zip(premise_sentences, embedded_premise):
        similarity = F.cosine_similarity(embedded_claim.last_hidden_state.mean(dim=1),
                                         premise_embedding.last_hidden_state.mean(dim=1), dim=1)

        # Check for the highest similarity
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_premise_sentence = sentence

    # Create a dictionary for the current claim
    claim_data = {
        "context": passage,
        "claim" : claim,
        "verdict": label,  # Replace with actual content from column 2
        "evidence": best_premise_sentence   # Replace with actual content from column 3
    }

    # Append the claim data to the output list
    output_data.append(claim_data)

    i = i + 1
    print(i)

# Write the output data to a JSON file
with open('data.json', 'w') as jsonfile:
    json.dump(output_data, jsonfile, indent=4)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

{'input_ids': tensor([[  101, 72337, 72654, 10911, 10155, 11178, 15541, 13028,   112,   172,
         11850,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
